Your markdown or comments here

In [ ]:
#pip install selenium webdriver-manager
import os
import time
import requests
import yt_dlp
from bs4 import BeautifulSoup
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader, YoutubeLoader, UnstructuredPowerPointLoader 
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import GitLoader
from langchain_community.vectorstores import Chroma

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, TimeoutException, StaleElementReferenceException

from webdriver_manager.chrome import ChromeDriverManager

from urllib.parse import urljoin, urlparse

load_dotenv()


In [ ]:
# WEB CRAWLER
def crawl_internal_links(start_url, max_pages=50, max_depth=1):
    """
    Crawl internal URLs from a site using Selenium, with support for JavaScript-heavy pages.

    Args:
        start_url (str): The URL to start crawling from.
        max_pages (int): Max number of pages to crawl.
        max_depth (int): Max depth to crawl (0 = just root).

    Returns:
        list: A list of internal URLs that were successfully visited.
    """
    # Selenium Headless Browser Setup
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    visited = set()
    domain = urlparse(start_url).netloc
    to_visit = [(start_url, 0)]

    while to_visit and len(visited) < max_pages:
        url, depth = to_visit.pop(0)
        if url in visited or depth > max_depth:
            continue

        try:
            driver.get(url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "a")))
            time.sleep(1)  # 🔁 Wait for JS to load

            print(f"Visited ({len(visited)+1}/{max_pages}), Depth {depth}): {url}")
            visited.add(url)

            # If max depth reached, skip link extraction
            if depth == max_depth:
                continue

            # Extract and queue internal links
            links = driver.find_elements(By.TAG_NAME, "a")
            for link in links:
                try:
                    href = link.get_attribute("href")
                    if not href or href.startswith(("mailto:", "tel:", "javascript:")):
                        continue

                    parsed = urlparse(href)
                    if parsed.netloc == domain or parsed.netloc == "":
                        full_url = urljoin(url, href).split("#")[0]
                        if full_url not in visited and all(full_url != q[0] for q in to_visit):
                            to_visit.append((full_url, depth + 1))
                except StaleElementReferenceException:
                    continue

        except (WebDriverException, TimeoutException):
            print(f"⚠️ Skipping (Error): {url}")
            visited.add(url)
            continue

    driver.quit()

    # Print summary
    print(f"\n Total unique internal URLs visited: {len(visited)}")

    if len(visited) < max_pages:
        print("⚠️ Number of crawled URLs is less than max_pages. Possible reasons:")
        print("- Site may not have enough unique pages within the allowed depth.")
        print("- Some links might be hidden behind JavaScript interactions.")
        print("- Some links could be blocked, inaccessible, or slow-loading.")
        print("- Your max_depth may be too shallow to discover deeper links, try changing depth.")
    
    return list(visited)

# **************** CONFIGURABLE SETTINGS *********************
start_url = "https://fidesinnova.io/"
max_pages = 53   # Maximum number of pages to visit
max_depth = 2    # 0 = only root, 1 = root + links from root

#  Run the Crawler
web_docs_list1 = crawl_internal_links(start_url, max_pages, max_depth)

#  Load the web documents
web_docs1 = []
for idx in web_docs_list1:
    a = WebBaseLoader(idx)
    try:
        web_docs1.extend(a.load())
        print(idx + " is loaded.")
    except:
        print(f"{idx} is not loaded.")

In [ ]:
# --- Step 2: Load environment variables from .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
# Headless setup
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--log-level=3")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url = "https://www.fidesinnova.io/"
visited = set()
web_docs_list2 = set()

def is_valid_internal(url):
    parsed = urlparse(url)
    return parsed.scheme in ["http", "https"] and parsed.netloc.endswith("fidesinnova.io")

def crawl(url):
    if url in visited:
        return
    visited.add(url)

    try:
        driver.get(url)
        time.sleep(2)  # Let dynamic content load

        links = driver.find_elements("tag name", "a")
        for link in links:
            href = link.get_attribute("href")
            if href:
                full_url = urljoin(url, href.split("#")[0])  # remove fragments
                if is_valid_internal(full_url) and full_url not in web_docs_list2:
                    web_docs_list2.add(full_url)
                    crawl(full_url)

    except Exception as e:
        print(f"Error visiting {url}: {e}")

# Start
crawl(base_url)

driver.quit()

# Output
print("\n=== Discovered Internal URLs ===")
for url in sorted(web_docs_list2):
    print(url)
print(f"\nTotal internal URLs web_docs_list2: {len(web_docs_list2)}")


#  Load the web documents
web_docs2 = []
for idx in web_docs_list2:
    a = WebBaseLoader(idx)
    try:
        web_docs2.extend(a.load())
        print(idx + " is loaded.")
    except:
        print(f"{idx} is not loaded.")


In [ ]:
# # Access the token
# GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

# HEADERS = {
#     "User-Agent": "Mozilla/5.0",
#     "Accept": "application/vnd.github.v3+json",
#     "Authorization": f"token {GITHUB_TOKEN}"
# }

# readable_extensions = {
#     ".py", ".json", ".html", ".js", ".ts", ".css", ".java",
#     ".c", ".cpp", ".rs", ".txt", ".md", ".yml", ".yaml", ".sh"
# }

# def is_readable_file(filename):
#     return any(filename.endswith(ext) for ext in readable_extensions)

# def get_default_branch(owner, repo):
#     url = f"https://api.github.com/repos/{owner}/{repo}"
#     try:
#         response = requests.get(url, headers=HEADERS, timeout=10)
#         if response.status_code == 200:
#             return response.json().get("default_branch", "main")
#         else:
#             print(f"❌ Failed to get default branch for {owner}/{repo} ({response.status_code})")
#             return "main"
#     except requests.exceptions.RequestException as e:
#         print(f"❌ Exception getting default branch for {owner}/{repo}: {e}")
#         return "main"

# def list_files_from_api(owner, repo, branch, path="", depth=0, max_depth=10):
#     url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}?ref={branch}"
#     try:
#         response = requests.get(url, headers=HEADERS, timeout=15)
#     except requests.exceptions.RequestException as e:
#         print(f"❌ Exception accessing {url}: {e}")
#         return []

#     if response.status_code == 403:
#         print(f"❌ Rate limit hit or access denied for: {url}")
#         return []
#     elif response.status_code != 200:
#         print(f"❌ Failed to access: {url} ({response.status_code})")
#         return []

#     items = response.json()
#     files = []

#     # Defensive: sometimes API returns a dict on error
#     if isinstance(items, dict) and items.get("message"):
#         print(f"❌ API error at {url}: {items['message']}")
#         return []

#     for item in items:
#         if item["type"] == "file":
#             filename = item["name"]
#             if is_readable_file(filename):
#                 files.append(item["path"])
#         elif item["type"] == "dir" and depth < max_depth:
#             # Debug print to track recursion progress:
#             print(f"🔍 Exploring directory: {item['path']} (depth {depth+1})")
#             files += list_files_from_api(owner, repo, branch, item["path"], depth+1, max_depth)

#     return files

# github_repos = [
#     "TheArchitect2000/Fides-Innova-WiKi",
#     "TheArchitect2000/Blockchain-based-IoT-Server",
#     "TheArchitect2000/ZKP-IoT-Arm-Siemens-IoT2050-C",
#     "TheArchitect2000/zkiot-riscv-qemu-c", 
#     "TheArchitect2000/Fides-Innova-Smart-Contract-Protocol",
#     "TheArchitect2000/ZKP-Blockchain-Explorer",
#     "TheArchitect2000/evm-server",
#     "TheArchitect2000/New-IoT-Device-Integration",
#     "TheArchitect2000/zkiot-riscv-qemu-rust"
# ]

# github_docs = set()
# for full_name in github_repos:
#     owner, repo = full_name.split("/")
#     print(f"\n📦 Crawling repository: {full_name}")
#     default_branch = get_default_branch(owner, repo)
#     files = list_files_from_api(owner, repo, default_branch)
#     if files:
#         print(f"✅ Found {len(files)} readable files:")
#         for f in files:
#             t1 = f"https://github.com/{full_name}/blob/main/{f}"
#             print("   -", t1)
#             github_docs.add(t1)
#     else:
#         print("⚠️ No readable files found or repo is empty/private.")


In [ ]:
# loading GitHub Repos
github_repos = [
    "https://github.com/TheArchitect2000/Fides-Innova-WiKi",
    "https://github.com/TheArchitect2000/Blockchain-based-IoT-Server",
    "https://github.com/TheArchitect2000/ZKP-IoT-Arm-Siemens-IoT2050-C",
    "https://github.com/TheArchitect2000/zkiot-riscv-qemu-c", 
    "https://github.com/TheArchitect2000/Fides-Innova-Smart-Contract-Protocol",
    "https://github.com/TheArchitect2000/ZKP-Blockchain-Explorer",
    "https://github.com/TheArchitect2000/evm-server",
    "https://github.com/TheArchitect2000/New-IoT-Device-Integration",
    "https://github.com/TheArchitect2000/zkiot-riscv-qemu-rust"
]

github_docs = []

for url in github_repos:
    print(f"📥 Loading repository: {url}")
    repo_name = url.split("/")[-1]
    local_path = f"./cloned_repos/{repo_name}"

    loader = GitLoader(
        repo_path=local_path,
        clone_url=url,
        branch="main",
        file_filter=lambda f: f.endswith((
            ".py", ".md", ".c", ".cpp", ".rs", ".json", ".html",
            ".js", ".ts", ".css", ".java", ".txt", ".yml", ".yaml", ".sh"
        ))
    )
    
    github_docs = loader.load()
    print(f"✅ Loaded {len(github_docs)} documents from {repo_name}")


In [ ]:
# --- Step 5: YouTube Channel Crawler ---

# Fides Innova YouTube Channel ID
CHANNEL_ID = "UCrrqGYx98H1dPdZsNb1i9-g"
CHANNEL_URL = f"https://www.youtube.com/channel/{CHANNEL_ID}"

def fetch_video_urls(channel_url: str):
    ydl_opts = {
        'ignoreerrors': True,
        'quiet': True,
        'extract_flat': True,  # Only metadata, not downloading
        'force_generic_extractor': False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(channel_url, download=False)
        video_urls = []
        video_ids = []

        if 'entries' in result:
            for entry in result['entries']:
                if entry and 'id' in entry:
                    video_url = f"https://www.youtube.com/watch?v={entry['id']}"
                    video_urls.append(video_url)
                    video_ids.append(entry['id'])
        return video_ids

# Fetch and display video URLs
video_ids = fetch_video_urls(CHANNEL_URL)
print(f"✅ Found {len(video_ids)} videos on channel.\n")

youtube_docs = []
second_try_idx = []
third_try_idx = []
for idx in video_ids:
    a = YoutubeLoader(idx)
    try:
        youtube_docs.extend(a.load())
        print(idx + " is loaded.")
    except:
        print(f"{idx} is not loaded.")
        second_try_idx.append(str(idx))
        print("\n unloaded list ")
        print(second_try_idx)
        print("\n")

print("\n")
print("second try for unloded videos.")

for idx in second_try_idx:
    print("waiting 10 seconds...")
    time.sleep(10)
    a = YoutubeLoader(idx)
    try:
        youtube_docs.extend(a.load())
        print(idx + " is loaded in the second try.")
    except:
        print("\n")
        print(f"{idx} is not loaded in the second try.")
        third_try_idx.append(str(idx))
        print("\n unloaded list for the second time ")
        print(second_try_idx)
        print("\n")

def change_YouTube_doc(doc):
    doc.metadata['type']='YouTube'
    return doc

youtube_docs = list(map(change_YouTube_doc, youtube_docs))

print("=== ALL youtube videos ===)")
print(len(youtube_docs))
print("=== Loaded youtube videos ===)")
print(len(youtube_docs)-len(third_try_idx))

In [ ]:
# --- Step 6: Load PDFs ---
pdf_docs = []
pdf_files = [
    "PDF/zkIoT.pdf",
]

for path in pdf_files:
    try:
        loader = PyPDFLoader(path)
        pdf_docs.extend(loader.load())
        print(len(pdf_docs))
    except Exception as e:
        print(f"Error loading PDF {path}: {e}")

def change_pdf_doc(doc):
    doc.metadata['type']='PDF'
    return doc

pdf_docs = list(map(change_pdf_doc, pdf_docs))

print("=== ALL PDF docs ===)")
print(len(pdf_docs))

In [ ]:

pptx_docs = []
pptx_files = [
    "PPTX/FidesinnovaDeck-v11.pptx"
]

for path in pptx_files:
    try:
        loader = UnstructuredPowerPointLoader(path)
        pptx_docs.extend(loader.load())
        print(f"✅ Loaded: {path}")
    except Exception as e:
        print(f"❌ Error loading PPTX {path}: {e}")

# Add metadata
def change_pptx_doc(doc):
    doc.metadata['type'] = 'PPTX'
    return doc

pptx_docs = list(map(change_pptx_doc, pptx_docs))

print("=== ALL PPTX docs ===)")
print(len(pptx_docs))

In [ ]:
# --- Step 7: Split & Vectorize ---
# all_docs =  youtube_docs + pdf_docs + pptx_docs + web_docs1 + web_docs2 + github_docs

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
split_docs1 = splitter.split_documents(youtube_docs)
split_docs2 = splitter.split_documents(pdf_docs)
split_docs3 = splitter.split_documents(pptx_docs)
split_docs4 = splitter.split_documents(web_docs1)
split_docs5 = splitter.split_documents(web_docs2)
split_docs6 = splitter.split_documents(github_docs)

embedding = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    collection_name="fides_crawled_data",
    embedding_function=embedding,
    persist_directory="./chroma_langchain_db"
)

vectordb.add_documents(split_docs1)
vectordb.add_documents(split_docs2)
vectordb.add_documents(split_docs3)
vectordb.add_documents(split_docs4)
vectordb.add_documents(split_docs5)

# Instead of adding GitHun large documents using:
# vectordb.add_documents(split_docs6)
# we do this:
batch_size = 100 
for i in range(0, len(split_docs6), batch_size):
    batch = split_docs6[i:i + batch_size]
    vectordb.add_documents(batch)
    print(f"✅ Added batch {i // batch_size + 1} of {len(split_docs6)/batch_size} documents")

print("✅ All documents crawled, split, and stored in vector DB.")

In [ ]:
# Instead of adding GitHun large documents using:
# vectordb.add_documents(split_docs6)
# we do this:
batch_size = 100 
for i in range(0, len(split_docs6), batch_size):
    batch = split_docs6[i:i + batch_size]
    vectordb.add_documents(batch)
    print(f"✅ Added batch {i // batch_size + 1} of {len(split_docs6)/batch_size} documents")

print("✅ All documents crawled, split, and stored in vector DB.")

In [ ]:
len(vectordb)